In [1]:
%%configure
{"conf": {
    "spark.app.name": "dslab-group_final"
}}

ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
4557,application_1587988164357_1185,pyspark,idle,Link,Link,
4558,application_1587988164357_1186,pyspark,idle,Link,Link,
4561,application_1587988164357_1189,pyspark,idle,Link,Link,
4562,application_1587988164357_1190,pyspark,idle,Link,Link,
4564,application_1587988164357_1192,pyspark,idle,Link,Link,
4565,application_1587988164357_1193,pyspark,busy,Link,Link,


In [2]:
from pyspark.sql.functions import *

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
4566,application_1587988164357_1194,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
# Load each day of typical week into spark dataframe
days_dict = {0: 'Monday', 1: 'Tuesday', 2: 'Wednesday', 3: 'Thursday', 4: 'Friday'}
d = 13
stop_times = (spark.read.format('csv').options(header='true', inferSchema='true')
              .load("/data/sbb/timetables/csv/stop_times/2019/05/%s/stop_times.txt" % d))
stops = (spark.read.format('csv').options(header='true', inferSchema='true')
         .load("/data/sbb/timetables/csv/stop_times/2019/05/%s/stops.txt" % d))
for d in range(14, 18):
    stop_times_d = (spark.read.format('csv').options(header='true', inferSchema='true')
                    .load("/data/sbb/timetables/csv/stop_times/2019/05/%s/stop_times.txt" % d))
    stop_times_d = stop_times_d.withColumn("day", lit(d))
    stop_times = stop_times.union(stop_times_d)
    
    stops_d = (spark.read.format('csv').options(header='true', inferSchema='true')
               .load("/data/sbb/timetables/csv/stop_times/2019/05/%s/stops.txt" % d))
    stop = stops.withColumn(withColumn("day", lit(d)))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
u'Path does not exist: hdfs://iccluster044.iccluster.epfl.ch:8020/data/sbb/timetables/csv/stop_times/2019/05/13/stop_times.txt;'
Traceback (most recent call last):
  File "/hdata/sdf/hadoop/yarn/local/usercache/ebouille/appcache/application_1587988164357_1176/container_e05_1587988164357_1176_01_000001/pyspark.zip/pyspark/sql/readwriter.py", line 166, in load
    return self._df(self._jreader.load(path))
  File "/hdata/sdf/hadoop/yarn/local/usercache/ebouille/appcache/application_1587988164357_1176/container_e05_1587988164357_1176_01_000001/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1257, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/hdata/sdf/hadoop/yarn/local/usercache/ebouille/appcache/application_1587988164357_1176/container_e05_1587988164357_1176_01_000001/pyspark.zip/pyspark/sql/utils.py", line 69, in deco
    raise AnalysisException(s.split(': ', 1)[1], stackTrace)
AnalysisException: u'Path does not exist: hdfs://i

In [3]:
stop_times = (spark.read.format('csv').options(header='true', inferSchema='true')
              .load("/data/sbb/timetables/csv/stop_times/2019/05/14/stop_times.txt"))
stops = (spark.read.format('csv').options(header='true', inferSchema='true')
         .load("/data/sbb/timetables/csv/stops/2019/05/14/stops.txt"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
stops.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------------------+----------------+-----------------+-------------+--------------+
|stop_id|           stop_name|        stop_lat|         stop_lon|location_type|parent_station|
+-------+--------------------+----------------+-----------------+-------------+--------------+
|1322000|            Altoggio|46.1672513851495|   8.345807131427|         null|          null|
|1322001|        Antronapiana| 46.060121674738| 8.11361957990831|         null|          null|
|1322002|              Anzola|45.9898698225697| 8.34571729989858|         null|          null|
|1322003|              Baceno|46.2614983591677| 8.31925293162473|         null|          null|
|1322004|Beura Cardezza, c...|46.0790618438814| 8.29927439970313|         null|          null|
|1322005|Bognanco, T. Vill...|46.1222963432243| 8.21077237789936|         null|          null|
|1322006|           Boschetto|46.0656504576122| 8.26113193273411|         null|          null|
|1322007|            Cadarese|46.2978807772998|  8

In [5]:
stop_times.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------------+--------------+-----------+-------------+-----------+-------------+
|             trip_id|arrival_time|departure_time|    stop_id|stop_sequence|pickup_type|drop_off_type|
+--------------------+------------+--------------+-----------+-------------+-----------+-------------+
|1.TA.1-1-B-j19-1.1.R|    04:20:00|      04:20:00|8500010:0:3|            1|          0|            0|
|1.TA.1-1-B-j19-1.1.R|    04:24:00|      04:24:00|8500020:0:3|            2|          0|            0|
|1.TA.1-1-B-j19-1.1.R|    04:28:00|      04:28:00|8500021:0:5|            3|          0|            0|
|1.TA.1-1-B-j19-1.1.R|    04:30:00|      04:30:00|8517131:0:2|            4|          0|            0|
|1.TA.1-1-B-j19-1.1.R|    04:32:00|      04:32:00|8500300:0:5|            5|          0|            0|
|1.TA.1-1-B-j19-1.1.R|    04:35:00|      04:35:00|8500313:0:2|            6|          0|            0|
|1.TA.1-1-B-j19-1.1.R|    04:37:00|      04:38:00|8500301:0:3|           

In [4]:
import networkx as nx
from geopy.distance import distance as geo_distance
from pyspark.sql.types import FloatType
from pyspark.sql import Row

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
zurich_pos = stops.where(column('stop_id')=='8503000').select('stop_lat', 'stop_lon').collect()
zurich_pos = (zurich_pos[0][0], zurich_pos[0][1])

def zurich_distance(x, y):
    return geo_distance(zurich_pos, (x,y)).km

stops_distance = stops.rdd.map(lambda x: (x['stop_id'], zurich_distance(x['stop_lat'], x['stop_lon'])))
stops_distance = spark.createDataFrame(stops_distance.map(lambda r: Row(stop_id=r[0], zurich_distance=r[1])))
stops_distance = stops_distance.filter(column('zurich_distance') <= 15)
stops_zurich = stops_distance.join(stops, on='stop_id')
print('Number of stations around Zurich:', stops_zurich.count())
stops_zurich.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

('Number of stations around Zurich:', 1880)
+-----------+------------------+--------------------+----------------+----------------+-------------+--------------+
|    stop_id|   zurich_distance|           stop_name|        stop_lat|        stop_lon|location_type|parent_station|
+-----------+------------------+--------------------+----------------+----------------+-------------+--------------+
|    8500926|11.510766966884365|Oetwil a.d.L., Sc...|47.4236270123012| 8.4031825286317|         null|          null|
|    8502186|10.798985488832079|Dietikon Stoffelbach|47.3934058321612|8.39894248049007|         null|      8502186P|
|8502186:0:1|10.800041577194426|Dietikon Stoffelbach|47.3934666445388|8.39894248049007|         null|      8502186P|
|8502186:0:2|10.801101793198619|Dietikon Stoffelbach|47.3935274568464|8.39894248049007|         null|      8502186P|
|   8502186P|10.798985488832079|Dietikon Stoffelbach|47.3934058321612|8.39894248049007|            1|          null|
|    8502187|12.4092

In [6]:
for r in stops_zurich.collect():
    if r['stop_id'] == '8503000':
        print((r['stop_id'], {'name': r['stop_name'].encode('utf-8'), 'lat': r['stop_lat'], 'lon': r['stop_lon']}))
        break

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(u'8503000', {'lat': 47.3781762039461, 'lon': 8.54019357578468, 'name': 'Z\xc3\xbcrich HB'})

In [7]:
graph = nx.MultiDiGraph()
nodes = stops_zurich.rdd.map(lambda r: (r[0], {'name': r['stop_name'],
                                              'lat': r['stop_lat'],
                                              'lon': r['stop_lon']})).collect()
graph.add_nodes_from(nodes)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
@udf
def convertToMinute(s):
    h, m, _ = s.split(':')
    h,m = int(h), int(m)
    
    return h*60+m

# Need to have one day only or filter on days (trip_id unique in a day)
nodes_list = list(graph.nodes())
stop_times_zurich = stop_times.filter(column('stop_id').isin(nodes_list))
stop_times_zurich = stop_times_zurich.filter(column('stop_id').isin(list(graph.nodes())))
stop_times_zurich = stop_times_zurich.withColumn('arrival_minute', convertToMinute(column('arrival_time')))
stop_times_zurich = stop_times_zurich.withColumn('departure_minute', convertToMinute(column('departure_time')))
stop_times_zurich_2 = (stop_times_zurich.withColumn('stop_sequence_prev', column('stop_sequence')-1)
                .select('trip_id',
                        column('stop_id').alias('next_stop'),
                        column('stop_sequence_prev').alias('stop_sequence'),
                        column('arrival_minute').alias('next_arrival_minute')))
edges_df = stop_times_zurich.join(stop_times_zurich_2, on=['trip_id', 'stop_sequence']).orderBy('trip_id', 'stop_sequence')
edges_df = edges_df.withColumn('trip_duration', column('next_arrival_minute')-column('departure_minute'))
edges_df = edges_df.select('stop_id', 'arrival_minute', 'departure_minute', 'next_stop', 'trip_duration')
edges_df.show()

edges = edges_df.rdd.map(lambda r: (r['stop_id'], r['next_stop'], {'duration': r['trip_duration'],
                                                          'time': r['departure_minute']})).collect()
print('Number of edges:', len(edges))
graph.add_edges_from(edges)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
Invalid status code '400' from http://iccluster044.iccluster.epfl.ch:8998/sessions/4566/statements/7 with error payload: "requirement failed: Session isn't active."
